In [67]:
import numpy as np
import pandas as pd
import xlrd

import sys
import os
folder_name = '历史记录'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

import shutil
      
def copyDirectory(src, dest):
    try:
        shutil.copytree(src, dest)
    # Directories are the same
    except shutil.Error as e:
        print('Directory not copied. Error: %s' % e)
    # Any error saying that the directory doesn't exist
    except OSError as e:
        print('Directory not copied. Error: %s' % e)
        
# copy historical summary data
month = int(sys.argv[1])
#month = 3
cwd = os.getcwd()
print(cwd)

folder_list = ['成品销售明细表','成品仓库进出明细']
term_list = ['品牌']

if (month == 1):
    pass
else:
    for term in term_list:
        for folder_name in folder_list:
            for i in range(1, month):
                if (i != 11 or i != 10):
                    src= cwd[:-1] + str(i) + '/'+term+'/'+folder_name
                    dest = cwd + '/历史记录/'+term+'/'+folder_name + '_0' + str(i)
                else:
                    src = cwd[:-2] + str(i) + '/'+term+'/'+folder_name
                    dest = cwd + '/历史记录/'+term+'/'+folder_name + '_' + str(i)
                    
                if not os.path.exists(dest):
                    copyDirectory(src, dest)

ValueError: invalid literal for int() with base 10: '-f'

In [23]:
# add the historical cumulative selling information
# example

history_folder_flag = False
if os.path.exists('历史记录'):
    history_folder_flag = True

cwd= os.getcwd()
if history_folder_flag:
    # get previous months information
    df_months = list()
    for check_month in range(1,month):
        key = '销售收入'
        if check_month <10:
            previous_folder_name = '成品销售明细表_0'+ str(check_month)
        else:
            previous_folder_name = '成品销售明细表_'+ str(check_month)

        input_filename = '历史记录/品牌/'+previous_folder_name+'/summary_'+key+'_品牌分类.xlsx'
        df_check_month = pd.read_excel(input_filename, sheet_name = 0)

        # rebuild dataframe
        term_names = list(df_check_month.iloc[0,:].dropna())
        #category_names = list(df_check_month.iloc[1,:].dropna().unique())
        #print(category_names)
        category_names = ['金额', '装车金额', '合计金额']
        new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
        df_check_month= df_check_month.drop([0,1,2])
        df_check_month.set_index(df_check_month.columns.values[0], inplace=True)
        df_check_month.columns = new_columns
        
        df_months.append(df_check_month)
    
    # current month information
    folder_name = '成品销售明细表'
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    df_months.append(df_current_month)

In [42]:
df_current_month = df_months[-1]
df_current_month

term              所有品牌                         维万家                     星晴  \
category           0金额  1装车金额        2合计金额     0金额 1装车金额   2合计金额      0金额   
销售收入 (3月)                                                                   
瓷片         1.51022e+06  859.8  1.51108e+06  260822  14.1  260836   552108   
地砖              121830   70.2       121901       0     0       0  84821.6   
圣陶居            97779.4      0      97779.4       0     0       0        0   
欧驰               35580      0        35580       0     0       0        0   
浮雕                7664      0         7664       0     0       0        0   
运费                7370      0         7370       0     0       0        0   
腰线              4439.4      0       4439.4       0     0       0        0   
花片                2820      0         2820       0     0       0        0   
销售费用            1809.1      0       1809.1       0     0       0        0   
东晟                1798      0         1798       0     0       0        0   
木纹砖            1503.92    1.2      1505.12       0     0       0        0   
换包                 594      0          594       0     0       0        0   
木托                 195      0          195       0     0       0        0   
纸箱                 175      0          175       0     0       0        0   
镜面砖                165      3          168       0     0       0        0   
总计         1.79395e+06  934.2  1.79488e+06  260822  14.1  260836   636930   

term                           恒希   ...         佛山       出口                 \
category  1装车金额    2合计金额      0金额   ...      2合计金额      0金额 1装车金额    2合计金额   
销售收入 (3月)                           ...                                      
瓷片          6.9   552115   641891   ...          0  55323.6     0  55323.6   
地砖          0.9  84822.5  37008.9   ...          0        0     0        0   
圣陶居           0        0  11150.2   ...    85150.8        0     0        0   
欧驰            0        0        0   ...      35580        0     0        0   
浮雕            0        0     7664   ...          0        0     0        0   
运费            0        0        0   ...          0        0     0        0   
腰线            0        0   3896.4   ...        543        0     0        0   
花片            0        0     2300   ...        520        0     0        0   
销售费用          0        0        0   ...          0        0     0        0   
东晟            0        0        0   ...       1798        0     0        0   
木纹砖           0        0  1503.92   ...          0        0     0        0   
换包            0        0        0   ...          0        0     0        0   
木托            0        0        0   ...          0      195     0      195   
纸箱            0        0      175   ...          0        0     0        0   
镜面砖           0        0      165   ...          0        0     0        0   
总计          7.8   636937   705754   ...     123592  55518.6     0  55518.6   

term            无                   未注明                
category      0金额 1装车金额   2合计金额     0金额 1装车金额   2合计金额  
销售收入 (3月)                                              
瓷片              0     0       0       0     0       0  
地砖              0     0       0       0     0       0  
圣陶居        1478.4     0  1478.4       0     0       0  
欧驰              0     0       0       0     0       0  
浮雕              0     0       0       0     0       0  
运费           7370     0    7370       0     0       0  
腰线              0     0       0       0     0       0  
花片              0     0       0       0     0       0  
销售费用            0     0       0  1809.1     0  1809.1  
东晟              0     0       0       0     0       0  
木纹砖             0     0       0       0     0       0  
换包            594     0     594       0     0       0  
木托              0     0       0       0     0       0  
纸箱              0     0       0       0     0       0  
镜面砖             0     0       0       0     0       0  
总计         9442.4     0  9442.4  1809.1     0

In [36]:
#expenses_last_month = df_last_month.loc[:,(slice(None),'金额')]
#expenses_current_month = df_current_month.loc[:,(slice(None),'金额')]

#expenses_last_month = df_last_month.xs('合计金额', level='category', axis=1)
#expenses_current_month = df_current_month.xs('合计金额', level='category', axis=1)
#print(expenses_last_month.shape)
#print(expenses_current_month.shape)

expenses_months = list()
for i in range(0, len(df_months)):
    expenses_months.append( df_months[i].xs('合计金额', level='category', axis=1) )
    print(i+1, expenses_months[i].shape)
    

1 (14, 9)
2 (13, 9)
3 (16, 9)


In [37]:
#expenses of current month
expenses_months[-1]

term,所有品牌,维万家,星晴,恒希,恒希样板,佛山,出口,无,未注明
销售收入 (3月),,,,,,,,,
瓷片,1.51108e+06,260836,552115,642729,79.1,0,55323.6,0,0
地砖,121901,0,84822.5,37078.2,0,0,0,0,0
圣陶居,97779.4,0,0,11150.2,0,85150.8,0,1478.4,0
欧驰,35580,0,0,0,0,35580,0,0,0
浮雕,7664,0,0,7664,0,0,0,0,0
运费,7370,0,0,0,0,0,0,7370,0
腰线,4439.4,0,0,3896.4,0,543,0,0,0
花片,2820,0,0,2300,0,520,0,0,0
销售费用,1809.1,0,0,0,0,0,0,0,1809.1


In [38]:
expenses_cumulate = (pd.concat(expenses_months, sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

expenses_cumulate

,所有品牌,恒希,星晴,佛山,维万家,恒希样板,出口,无,未注明
总计,3181170.16,1562337.62,936208.54,125170.38,406374.08,169.9,130126.2,13836.64,6946.8
瓷片,2784588.92,1431664.50,817412.44,0.00,406374.08,116.7,129021.2,0.00,0.0
地砖,218388.60,99539.30,118796.10,0.00,0.00,53.2,0.0,0.00,0.0
圣陶居,103889.80,17260.60,0.00,85150.80,0.00,0.0,0.0,1478.40,0.0
欧驰,35579.98,0.00,0.00,35579.98,0.00,0.0,0.0,0.00,0.0
浮雕,12208.00,11984.00,0.00,224.00,0.00,0.0,0.0,0.00,0.0
运费,8771.12,0.00,0.00,0.00,0.00,0.0,0.0,8771.12,0.0
腰线,7436.60,6496.60,0.00,940.00,0.00,0.0,0.0,0.00,0.0
销售费用,6946.80,0.00,0.00,0.00,0.00,0.0,0.0,0.00,6946.8
镜面砖,5422.50,5422.50,0.00,0.00,0.00,0.0,0.0,0.00,0.0


In [39]:
columns_names = list(expenses_cumulate)
new_columns = pd.MultiIndex.from_product([columns_names, ['3累计合计金额']], names=['term', 'category'])
expenses_cumulate.columns = new_columns
expenses_cumulate

term,所有品牌,恒希,星晴,佛山,维万家,恒希样板,出口,无,未注明
category,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额,3累计合计金额
总计,3181170.16,1562337.62,936208.54,125170.38,406374.08,169.9,130126.2,13836.64,6946.8
瓷片,2784588.92,1431664.50,817412.44,0.00,406374.08,116.7,129021.2,0.00,0.0
地砖,218388.60,99539.30,118796.10,0.00,0.00,53.2,0.0,0.00,0.0
圣陶居,103889.80,17260.60,0.00,85150.80,0.00,0.0,0.0,1478.40,0.0
欧驰,35579.98,0.00,0.00,35579.98,0.00,0.0,0.0,0.00,0.0
浮雕,12208.00,11984.00,0.00,224.00,0.00,0.0,0.0,0.00,0.0
运费,8771.12,0.00,0.00,0.00,0.00,0.0,0.0,8771.12,0.0
腰线,7436.60,6496.60,0.00,940.00,0.00,0.0,0.0,0.00,0.0
销售费用,6946.80,0.00,0.00,0.00,0.00,0.0,0.0,0.00,6946.8


In [43]:
df_current_month.rename(index=str, columns={'金额': '0金额', '装车金额': '1装车金额', '合计金额':'2合计金额'}, inplace=True)

new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
new_df.sort_index(axis=1, inplace=True)
new_df.rename(index=str, columns={'0金额': '金额', '1装车金额': '装车金额', '2合计金额':'合计金额',
                                  '3累计合计金额': '累计合计金额'}, inplace=True)
new_df = new_df.sort_values(by=('所有品牌','累计合计金额'),ascending = False)
new_df

term             佛山                                  出口                \
category         金额 装车金额       合计金额     累计合计金额       金额 装车金额     合计金额   
总计        123591.78  0.0  123591.78  125170.38  55518.6  0.0  55518.6   
瓷片             0.00  0.0       0.00       0.00  55323.6  0.0  55323.6   
地砖             0.00  0.0       0.00       0.00      0.0  0.0      0.0   
圣陶居        85150.80  0.0   85150.80   85150.80      0.0  0.0      0.0   
欧驰         35579.98  0.0   35579.98   35579.98      0.0  0.0      0.0   
浮雕             0.00  0.0       0.00     224.00      0.0  0.0      0.0   
运费             0.00  0.0       0.00       0.00      0.0  0.0      0.0   
腰线           543.00  0.0     543.00     940.00      0.0  0.0      0.0   
销售费用           0.00  0.0       0.00       0.00      0.0  0.0      0.0   
镜面砖            0.00  0.0       0.00       0.00      0.0  0.0      0.0   
花片           520.00  0.0     520.00     520.00      0.0  0.0      0.0   
换包             0.00  0.0       0.00       0.00      0.0  0.0      0.0   
东晟          1798.00  0.0    1798.00    2755.60      0.0  0.0      0.0   
木纹砖            0.00  0.0       0.00       0.00      0.0  0.0      0.0   
纸箱             0.00  0.0       0.00       0.00      0.0  0.0      0.0   
其它费用           0.00  0.0       0.00       0.00      0.0  0.0      0.0   
换托费            0.00  0.0       0.00       0.00      0.0  0.0      0.0   
木托             0.00  0.0       0.00       0.00    195.0  0.0    195.0   
捆砖带            0.00  0.0       0.00       0.00      0.0  0.0      0.0   

term                       恒希           ...            星晴                未注明  \
category    累计合计金额         金额   装车金额    ...          合计金额     累计合计金额      金额   
总计        130126.2  705754.02  912.3    ...      636937.4  936208.54  1809.1   
瓷片        129021.2  641890.60  838.8    ...      552114.9  817412.44     0.0   
地砖             0.0   37008.90   69.3    ...       84822.5  118796.10     0.0   
圣陶居            0.0   11150.20    0.0    ...           0.0       0.00     0.0   
欧驰             0.0       0.00    0.0    ...           0.0       0.00     0.0   
浮雕             0.0    7664.00    0.0    ...           0.0       0.00     0.0   
运费             0.0       0.00    0.0    ...           0.0       0.00     0.0   
腰线             0.0    3896.40    0.0    ...           0.0       0.00     0.0   
销售费用           0.0       0.00    0.0    ...           0.0       0.00  1809.1   
镜面砖            0.0     165.00    3.0    ...           0.0       0.00     0.0   
花片             0.0    2300.00    0.0    ...           0.0       0.00     0.0   
换包             0.0       0.00    0.0    ...           0.0       0.00     0.0   
东晟             0.0       0.00    0.0    ...           0.0       0.00     0.0   
木纹砖            0.0    1503.92    1.2    ...           0.0       0.00     0.0   
纸箱             0.0     175.00    0.0    ...           0.0       0.00     0.0   
其它费用           0.0       0.00    0.0    ...           0.0       0.00     0.0   
换托费            0.0       0.00    0.0    ...           0.0       0.00     0.0   
木托          1105.0       0.00    0.0    ...           0.0       0.00     0.0   
捆砖带            0.0       0.00    0.0    ...           0.0       0.00     0.0   

term                                 维万家                              
category 装车金额    合计金额  累计合计金额         金额  装车金额       合计金额     累计合计金额  
总计        0.0  1809.1  6946.8  260821.88  14.1  260835.98  406374.08  
瓷片        0.0     0.0     0.0  260821.88  14.1  260835.98  406374.08  
地砖        0.0     0.0     0.0       0.00   0.0       0.00       0.00  
圣陶居       0.0     0.0     0.0       0.00   0.0       0.00       0.00  
欧驰        0.0     0.0     0.0       0.00   0.0       0.00       0.00  
浮雕        0.0     0.0     0.0       0.00   0.0       0.00       0.00  
运费        0.0     0.0     0.0       0.00   0.0       0.00       0.00  
腰线        0.0     0.0     0.0       0.00   0.0       0.00       0.00  
销售费用      0.0  1809.1  6946.8       0.00   0.0       0.00       0.00  
镜面砖 

In [44]:
N_col= new_df.columns.levels[0].size
new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')
              
if '无' in new_columns:
    new_columns = new_columns.drop('无').insert(N_col,'无')
if '未注明' in new_columns:
    new_columns = new_columns.drop('未注明').insert(N_col,'未注明')
    
new_df = new_df.reindex(new_columns, level='term', axis=1)

N_row = new_df.shape[0]
new_rows = new_df.index.drop('总计').insert(N_row, '总计')
new_df = new_df.reindex(new_rows)

new_df = new_df.round(2)
new_df

term            所有品牌                                        佛山       \
category          金额   装车金额        合计金额      累计合计金额         金额 装车金额   
瓷片        1510223.18  859.8  1511082.98  2784588.92       0.00  0.0   
地砖         121830.50   70.2   121900.70   218388.60       0.00  0.0   
圣陶居         97779.40    0.0    97779.40   103889.80   85150.80  0.0   
欧驰          35579.98    0.0    35579.98    35579.98   35579.98  0.0   
浮雕           7664.00    0.0     7664.00    12208.00       0.00  0.0   
运费           7370.00    0.0     7370.00     8771.12       0.00  0.0   
腰线           4439.40    0.0     4439.40     7436.60     543.00  0.0   
销售费用         1809.10    0.0     1809.10     6946.80       0.00  0.0   
镜面砖           165.00    3.0      168.00     5422.50       0.00  0.0   
花片           2820.00    0.0     2820.00     4580.00     520.00  0.0   
换包            594.00    0.0      594.00     3372.00       0.00  0.0   
东晟           1798.00    0.0     1798.00     2755.60    1798.00  0.0   
木纹砖          1503.92    1.2     1505.12     1505.12       0.00  0.0   
纸箱            175.00    0.0      175.00      175.00       0.00  0.0   
其它费用            0.00    0.0        0.00      138.80       0.00  0.0   
换托费             0.00    0.0        0.00       76.32       0.00  0.0   
木托            195.00    0.0      195.00    -6175.00       0.00  0.0   
捆砖带             0.00    0.0        0.00    -8490.00       0.00  0.0   
总计        1793946.48  934.2  1794880.68  3181170.16  123591.78  0.0   

term                                 出口        ...          维万家             \
category       合计金额     累计合计金额       金额 装车金额   ...         合计金额     累计合计金额   
瓷片             0.00       0.00  55323.6  0.0   ...    260835.98  406374.08   
地砖             0.00       0.00      0.0  0.0   ...         0.00       0.00   
圣陶居        85150.80   85150.80      0.0  0.0   ...         0.00       0.00   
欧驰         35579.98   35579.98      0.0  0.0   ...         0.00       0.00   
浮雕             0.00     224.00      0.0  0.0   ...         0.00       0.00   
运费             0.00       0.00      0.0  0.0   ...         0.00       0.00   
腰线           543.00     940.00      0.0  0.0   ...         0.00       0.00   
销售费用           0.00       0.00      0.0  0.0   ...         0.00       0.00   
镜面砖            0.00       0.00      0.0  0.0   ...         0.00       0.00   
花片           520.00     520.00      0.0  0.0   ...         0.00       0.00   
换包             0.00       0.00      0.0  0.0   ...         0.00       0.00   
东晟          1798.00    2755.60      0.0  0.0   ...         0.00       0.00   
木纹砖            0.00       0.00      0.0  0.0   ...         0.00       0.00   
纸箱             0.00       0.00      0.0  0.0   ...         0.00       0.00   
其它费用           0.00       0.00      0.0  0.0   ...         0.00       0.00   
换托费            0.00       0.00      0.0  0.0   ...         0.00       0.00   
木托             0.00       0.00    195.0  0.0   ...         0.00       0.00   
捆砖带            0.00       0.00      0.0  0.0   ...         0.00       0.00   
总计        123591.78  125170.38  55518.6  0.0   ...    260835.98  406374.08   

term           无                            未注明                       
category      金额 装车金额    合计金额    累计合计金额      金额 装车金额    合计金额  累计合计金额  
瓷片           0.0  0.0     0.0      0.00     0.0  0.0     0.0     0.0  
地砖           0.0  0.0     0.0      0.00     0.0  0.0     0.0     0.0  
圣陶居       1478.4  0.0  1478.4   1478.40     0.0  0.0     0.0     0.0  
欧驰           0.0  0.0     0.0      0.00     0.0  0.0     0.0     0.0  
浮雕           0.0  0.0     0.0      0.00     0.0  0.0     0.0     0.0  
运费        7370.0  0.0  7370.0   8771.12     0.0  0.0     0.0     0.0  
腰线           0.0  0.0     0.0      0.00     0.0  0.0     0.0     0.0  
销售费用         0.0  0.0     0.0      0.00  1809.1  0.0  1809.1  6946.8  
镜面砖          0.0  0.0     0.0      0.00     0.0  0.0     0.0     0.0  
花片           0.0  0.0     0.0      0.00     0.0  0.0     0.0     0.0  
换包         594.0 

In [63]:
def create_cumulative_info_factory(month, key):
     # get previous months information
    folder_name = '成品仓库进出明细'
    df_months = list()
    for check_month in range(1,month):
        if check_month <10:
            previous_folder_name = folder_name+'_0'+ str(check_month)
        else:
            previous_folder_name = folder_name+'_'+ str(check_month)

        input_filename = '历史记录/品牌/'+previous_folder_name+'/summary_'+key+'_品牌分类.xlsx'
        df_check_month = pd.read_excel(input_filename, sheet_name = 0)

        # rebuild dataframe
        term_names = list(df_check_month.iloc[0,:].dropna())
        #category_names = list(df_check_month.iloc[1,:].dropna().unique())
        #print(category_names)
        category_names = ['数量(片)','金额']
        new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
        df_check_month= df_check_month.drop([0,1,2])
        df_check_month.set_index(df_check_month.columns.values[0], inplace=True)
        df_check_month.columns = new_columns
        
        df_months.append(df_check_month)
    
    # current month information
   
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['数量(片)','金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    df_months.append(df_current_month)  
    
    expenses_months = list()
    for i in range(0, len(df_months)):
        expenses_months.append( df_months[i].xs('金额', level='category', axis=1) )
        #print(i+1, expenses_months[i].shape)
    
    expenses_cumulate = (pd.concat(expenses_months, sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

    
    columns_names = list(expenses_cumulate)
    new_columns = pd.MultiIndex.from_product([columns_names, ['2累计金额']], names=['term', 'category'])
    expenses_cumulate.columns = new_columns
    
    df_current_month.rename(index=str, columns={'数量(片)':'0数量(片)', '金额': '1金额'}, inplace=True)

    new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
    new_df.sort_index(axis=1, inplace=True)
    new_df.rename(index=str, columns={'0数量(片)':'数量(片)', '1金额': '金额',
                                      '2累计金额':'累计金额(1-'+str(month)+'月)'}, inplace=True)
    
    new_df = new_df.sort_values(by=('所有品牌','累计金额(1-'+str(month)+'月)'),ascending = False)
    
    N_col= new_df.columns.levels[0].size
    new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')
    if '无' in new_columns:
        new_columns = new_columns.drop('无').insert(N_col,'无')
    new_df = new_df.reindex(new_columns, level='term', axis=1)

    N_row = new_df.shape[0]
    new_rows = new_df.index.drop('总计').insert(N_row, '总计')
    new_df = new_df.reindex(new_rows)

    new_df = new_df.round(2)
    return new_df



    

In [64]:
def create_cumulative_info_selling(month, key):
    folder_name = '成品销售明细表'
    # get previous months information
    df_months = list()
    for check_month in range(1,month):
        if check_month <10:
            previous_folder_name = folder_name + '_0'+ str(check_month)
        else:
            previous_folder_name = folder_name + '_'+ str(check_month)

        input_filename = '历史记录/品牌/'+previous_folder_name+'/summary_'+key+'_品牌分类.xlsx'
        df_check_month = pd.read_excel(input_filename, sheet_name = 0)

        # rebuild dataframe
        term_names = list(df_check_month.iloc[0,:].dropna())
        #category_names = list(df_check_month.iloc[1,:].dropna().unique())
        #print(category_names)
        category_names = ['金额', '装车金额', '合计金额']
        new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
        df_check_month= df_check_month.drop([0,1,2])
        df_check_month.set_index(df_check_month.columns.values[0], inplace=True)
        df_check_month.columns = new_columns
        
        df_months.append(df_check_month)
    
    # current month information
    input_filename = '品牌/'+folder_name+'/summary_'+key+'_品牌分类.xlsx'
    df_current_month = pd.read_excel(input_filename, sheet_name = 0)

    # rebuild dataframe
    term_names = list(df_current_month.iloc[0,:].dropna())
    category_names = ['金额', '装车金额', '合计金额']
    new_columns = pd.MultiIndex.from_product([term_names, category_names], names=['term', 'category'])
    df_current_month= df_current_month.drop([0,1,2])
    df_current_month.set_index(df_current_month.columns.values[0], inplace=True)
    df_current_month.columns = new_columns
    
    df_months.append(df_current_month)
    
    #extract expenses
    expenses_months = list()
    for i in range(0, len(df_months)):
        expenses_months.append( df_months[i].xs('合计金额', level='category', axis=1) )
        #print(i+1, expenses_months[i].shape)

    expenses_cumulate = (pd.concat(expenses_months, sort=False)
                     .groupby(level=0)
                     .sum()
                     .sort_values(by='所有品牌', ascending =False)
                    )

    columns_names = list(expenses_cumulate)
    new_columns = pd.MultiIndex.from_product([columns_names, ['3累计合计金额']], names=['term', 'category'])
    expenses_cumulate.columns = new_columns
    
    df_current_month.rename(index=str, columns={'金额': '0金额', '装车金额': '1装车金额',
                                                '合计金额':'2合计金额'}, inplace=True)

    new_df = pd.concat([df_current_month, expenses_cumulate], axis=1, sort=True).fillna(0.00)
    new_df.sort_index(axis=1, inplace=True)
    new_df.rename(index=str, columns={'0金额': '金额', '1装车金额': '装车金额', '2合计金额':'合计金额',
                                      '3累计合计金额': '累计合计金额(1-'+str(month)+'月)'}, inplace=True)
    new_df = new_df.sort_values(by=('所有品牌','累计合计金额(1-'+str(month)+'月)'),ascending = False)
    
    N_col= new_df.columns.levels[0].size
    new_columns = new_df.columns.levels[0].drop('所有品牌').insert(0,'所有品牌')

    if '无' in new_columns:
        new_columns = new_columns.drop('无').insert(N_col,'无')
    if '未注明' in new_columns:
        new_columns = new_columns.drop('未注明').insert(N_col,'未注明')

    new_df = new_df.reindex(new_columns, level='term', axis=1)

    N_row = new_df.shape[0]
    new_rows = new_df.index.drop('总计').insert(N_row, '总计')
    new_df = new_df.reindex(new_rows)

    new_df = new_df.round(2)
    return new_df



In [47]:
def handle_worksheet_format(writer, df_temp, title):
    workbook = writer.book
    worksheet = writer.sheets['sheet1']    
    
    for idx, col in enumerate(df_temp):  # loop through all columns
        series = df_temp[col]
        max_len = max(( series.astype(str).map(len).max() ,  # len of largest item
                        max( [len(str(x)) for x in series.name])*2  # len of column name/header
                      )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = df_temp.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name))  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)
    
    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})
    worksheet_cols = df_temp.shape[1]+1
    
    if (worksheet_cols <=25):
        range_str = 'A1:'+chr(ord('A')+worksheet_cols-1)+'1'
    else:
        last_col_str = chr(ord('A')+ worksheet_cols // 26 - 1) + chr(ord('A')+ (worksheet_cols-1) % 26) + '1'
        range_str = 'A1:'+last_col_str    
    worksheet.merge_range(range_str,title,header_format) 


In [58]:
def output_cumulative_info_dataframe(month, key, folder_name, out_path, title=None):
    if not os.path.exists(out_path + '/' + folder_name):
        os.makedirs(out_path + '/'+ folder_name)
        
    output_filename = out_path +'/'+ folder_name+'/统计_'+key+'_品牌分类.xlsx'      
    if title is None:
        month_str = ' ('+ str(month)+'月)'
        title=key + month_str
        
    if (folder_name == '成品仓库进出明细'):
        grouped =  create_cumulative_info_factory(month, key)
    elif (folder_name == '成品销售明细表'):
        grouped = create_cumulative_info_selling(month, key)
    else:
        print('Error for folder name')
        exit()
        
        
    # write to Excel file
    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workbook
    grouped.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook    
    
    
    # handle the spreadsheet format
    handle_worksheet_format(writer, grouped, title)       

    writer.save() #saves workbook to file in python file directory
    writer.close()
    
        

In [65]:
out_path = '累计金额信息'
folder_name = '成品仓库进出明细'

key_list = ['销售成本','免费样板','免费广告']
for key in key_list:
    output_cumulative_info_dataframe(month, key, folder_name, out_path)


In [66]:
out_path = '累计金额信息'
folder_name = '成品销售明细表'

key_list = ['销售收入', '挂账样板', '免费样板','免费广告','免费费用','破损赔偿','物资自用']
for key in key_list:
    output_cumulative_info_dataframe(month, key, folder_name, out_path)

